In [74]:
import pandas as pd
import json
import requests
import numpy as np
from datetime import datetime, timedelta
from pandas.io.json import json_normalize
from urllib.request import urlopen

In [77]:
# set dates to define week of data
in_date=pd.datetime.today()-timedelta(days=1)
start=in_date - timedelta(days=6)
in_date=in_date.strftime('%Y-%m-%d')
start=start.strftime('%Y-%m-%d')

In [78]:
date_list=pd.date_range(start, in_date, freq='D')
date_list=date_list.strftime('%Y-%m-%d')
date_list=date_list.astype(list)

In [25]:
#generate urls for each date in model
url_list=[]
for i in date_list:
    istr=str(i)
    url='https://raw.githubusercontent.com/alexlitel/congresstweets/master/data/'+istr+'.json'
    url_list.append(url)

In [8]:
#read in data
df_from_each_file = (pd.read_json(u) for u in url_list)
full=pd.concat(df_from_each_file, ignore_index=True)

In [9]:
#datetime conversion
full['dtime']=pd.to_datetime(full.time)

In [79]:
#get users file
usersurl='https://raw.githubusercontent.com/alexlitel/congresstweets-automator/master/data/users-filtered.json'

In [81]:
users=urlopen(usersurl).read()
users_dict=json.loads(users)

In [164]:
#extract office and campaign twitter handles
office = []
campaign = []
for j in range(0,len(users_dict)):
    try:
        if users_dict[j]['accounts'][0]['account_type']=='office':
            office.append(users_dict[j]['accounts'][0]['screen_name'])   
        elif users_dict[j]['accounts'][1]['account_type']=='office':
            office.append(users_dict[j]['accounts'][1]['screen_name'])
        elif users_dict[j]['accounts'][2]['account_type']=='office':
            office.append(users_dict[j]['accounts'][2]['screen_name'])
    except:
        office.append('empty')
    try:
        if users_dict[j]['accounts'][1]['account_type']=='campaign':
            campaign.append(users_dict[j]['accounts'][1]['screen_name'])
        elif users_dict[j]['accounts'][2]['account_type']=='campaign':
            campaign.append(users_dict[j]['accounts'][2]['screen_name'])
        elif users_dict[j]['accounts'][3]['account_type']=='campaign':
            campaign.append(users_dict[j]['accounts'][3]['screen_name'])
        elif users_dict[j]['accounts'][0]['account_type']=='campaign':
            campaign.append(users_dict[j]['accounts'][0]['screen_name'])
        elif users_dict[j]['accounts'][4]['account_type']=='campaign':
            campaign.append(users_dict[j]['accounts'][4]['screen_name'])
    except:
        campaign.append("empty")

In [148]:
handles=json_normalize(hand_dict)
handles.shape

(592, 7)

In [152]:
#create df series for handles
handles['officeSN']=office
handles['campaignSN']=campaign

In [165]:
#combine tweets with congressperson identifying info and create final df for week
temp1=pd.merge(full, handles, how='inner', left_on=['screen_name'], right_on=['officeSN'])
temp2=pd.merge(full, handles, how='inner', left_on=['screen_name'], right_on=['campaignSN'])

final = pd.concat([temp1,temp2])

In [160]:
#drop non individual twitter handles e.g. cloakroom handles
final=final[final.type=='member']

In [161]:
final=final.drop(['time'],axis=1)

In [168]:
final.text=final.text.str.lower()
final.text=final.text.str.replace('\n','')
final.text=final.text.str.replace('.','')

In [169]:
final.shape

(11359, 17)

In [170]:
fin.to_csv('tweet_save'+in_date+'.csv',date_format='%Y-%m-%d %H:%M:%S')